# Аналитика, связанная с зарплатами

## 1) Первый взгляд на данные

In [1]:
import pandas as pd
import plotly.express as px
import re

In [2]:
df = pd.read_csv('data/agg_data.csv')
print(df.shape)
print(df.columns)
display(df.head(3))
display(df.tail(3))
df.drop(columns='date_publish', inplace=True)

(5990, 13)
Index(['company', 'position', 'location', 'schedule', 'responsibilities',
       'requirements', 'levels', 'techstack', 'source', 'date_publish',
       'format', 'salary', 'currency'],
      dtype='object')


,company,position,location,schedule,responsibilities,requirements,levels,techstack,source,date_publish,format,salary,currency
0,Сбербанк,Аналитик данных,Москва,Полный рабочий день,не определено,не определено,От 1 года,"['sql', 'python', 'oracle', 'субд', 'greenplum...",superjob.ru,2022-12-25 10:50:27,Не имеет значения,Договорная,RUB
1,Friendwork,Data Engineer (remote),Москва,Полный рабочий день,не определено,не определено,От 3 лет,"['sql', 'python', 'postgresql', 'spark', 'dock...",superjob.ru,2022-12-19 00:00:00,Удалённая работа (на дому),Договорная,RUB
2,Технопарк «Сколково»,Teamlead проекта,Москва,Полный рабочий день,не определено,не определено,От 1 года,"['python', 'data science', 'bi', 'ios', 'c', '...",superjob.ru,2022-12-19 11:01:29,Не имеет значения,Договорная,RUB


,company,position,location,schedule,responsibilities,requirements,levels,techstack,source,date_publish,format,salary,currency
5987,не определено,не определено,не определено,[],не определено,• Высшее образование;\n• Опыт работы руковод...,[],"['java', 'c++', 'oracle']",https://t.me/s/datasciencejobs,2020-02-19 12:16:00,[],400000.0,RUB
5988,не определено,Data Scientist,не определено,['Fulltime'],не определено,"• 3 years of experience in business analysis, ...",[],['pandas'],https://t.me/s/datasciencejobs,2020-02-20 23:04:00,['Удалёнка'],3000.0,USD
5989,"Федеральная розничная сеть, занимающая лидирую...",Senior ML Engineer,Санкт-Петербург,[],- Проработать совместно с партнерами архитекту...,- Опыт построения/проектирования платформы для...,['Senior'],"['etl', 'linux']",https://t.me/s/datasciencejobs,2020-05-21 09:55:00,[],260000.0,RUB


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5990 entries, 0 to 5989
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   company           5990 non-null   object
 1   position          5990 non-null   object
 2   location          5990 non-null   object
 3   schedule          5990 non-null   object
 4   responsibilities  5990 non-null   object
 5   requirements      5990 non-null   object
 6   levels            5990 non-null   object
 7   techstack         5990 non-null   object
 8   source            5990 non-null   object
 9   format            5990 non-null   object
 10  salary            5990 non-null   object
 11  currency          5990 non-null   object
dtypes: object(12)
memory usage: 561.7+ KB


In [4]:
df['salary'].value_counts()

False            3249
True             1136
0                 765
не определено     252
300000.0           46
                 ... 
240000              1
120.0               1
437500              1
170500              1
40000               1
Name: salary, Length: 123, dtype: int64

In [5]:
df['salary'] =  pd.to_numeric(df['salary'], errors='coerce', downcast = 'float').fillna(0)
df['salary'].value_counts()

0.0         5415
250000.0      69
300000.0      65
200000.0      50
350000.0      45
            ... 
5500.0         1
800000.0       1
55000.0        1
315000.0       1
140000.0       1
Name: salary, Length: 93, dtype: int64

In [6]:
df[df['salary'] > 0].shape

(575, 12)

In [7]:
df['currency'].value_counts()

не определено    3501
RUB              2351
USD               111
EUR                26
AZN                 1
Name: currency, dtype: int64

In [8]:
df[df['salary'] > 0]['currency'].value_counts()

RUB    509
USD     56
EUR     10
Name: currency, dtype: int64

In [9]:
# преобразуем валюты:
df['salary'] = df.apply(lambda row: row['salary'] if row['currency'] == 'RUB' else (row['salary']*41  if row['currency'] == 'AZN' else row['salary']*70 ), axis = 1)             

In [10]:
def salary_range(x):
    step = 100000
    if x == 0:
        return 0
    for i in range(0,10):
        if x > i*step and x <= (i+1)*step:
            return (i+1)*step
        
df['salary_range'] = df['salary'].apply(salary_range)

In [51]:
px.histogram(df[df['salary'] > 0], x = 'salary_range', nbins = 10, color_discrete_sequence = ['steelblue'])

In [12]:
# посмотрим на зарпалты более 500к
df[df['salary'] > 500000]

,company,position,location,schedule,responsibilities,requirements,levels,techstack,source,format,salary,currency,salary_range
4516,NEWHR,Senior Ruby/RoR Developer for highload iPaaS p...,не определено,полный,не определено,не определено,Старший (Senior),"['Ruby on Rails', 'Высоконагруженные системы',...",habr.ru,удаленка,595000.0,RUB,600000.0
4588,Diabolocom,Java Developer,Not indicated,не указано,не определено,не определено,Старший (Senior),"['Английский язык', 'SQL', 'Java', 'Java Sprin...",habr.ru,не указано,700000.0,RUB,700000.0
4592,Reef Technologies,Senior Python Backend Engineer,не определено,не полный,не определено,не определено,Старший (Senior),"['Python', 'Проектирование баз данных']",habr.ru,удаленка,823200.0,RUB,900000.0
4594,Diabolocom,DevOps/SRE Engineer,Not indicated,не указано,не определено,не определено,Старший (Senior),"['Git', 'PostgreSQL', 'CI/CD', 'Kubernetes', '...",habr.ru,не указано,700000.0,RUB,700000.0
4819,EGGHEADS,Team Lead Backend (PHP),не определено,полный,не определено,не определено,Ведущий (Lead),"['SQL', 'PostgreSQL', 'MySQL', 'Yii framework'...",habr.ru,удаленка,550000.0,RUB,600000.0
5200,CentralPay,Senior Backend Developer (PHP/Laravel),не определено,полный,не определено,не определено,Старший (Senior),"['PHP', 'Laravel', 'Docker', 'Kubernetes']",habr.ru,удаленка,560000.0,RUB,600000.0
5445,ZERO (https://zerosystems.com),не определено,не определено,[],"- Design, build, and/or deliver NLP models for...",hugging_face: library\n- Excellent communicati...,"['Middle', 'Senior', 'Lead']","['jax', 'python', 'kubernetes', 'kubeflow', 'c...",https://t.me/s/datasciencejobs,[],700000.0,USD,700000.0
5488,КА SkillHunt (ищем в компанию разработчик моби...,"Computer Vision разработчик, уровня Senior, и ...",Ереван/Польша/Тайланд,[],• Ресерч.\n• Оптимизацию и дистилляцию нейросе...,• Опыт ML-разработки от 4-х лет.\n• Опыт работ...,"['Senior', 'Lead']",[],https://t.me/s/datasciencejobs,[],525000.0,USD,600000.0
5511,не определено,не определено,не определено,[],не определено,— уметь работать в команде: тестировать MVP на...,[],['pytorch'],https://t.me/s/datasciencejobs,"['Удалёнка', 'Офис']",700000.0,EUR,700000.0
5525,не определено,не определено,не определено,[],не определено,не определено,"['Middle', 'Senior']",[],https://t.me/s/datasciencejobs,[],7000000.0,USD,NaN


Некоторые компании публикуют одну и ту же вакансию в разных регионах. 
По сути это одна вакансия, поэтому мы будем рассматривать такие строки как дубликаты и удалять их
Такие вакансии отнесем к Москве.

In [13]:
ss = df.groupby(by = ['company', 'position', 'schedule', 'responsibilities',
       'requirements', 'levels', 'techstack', 'source',
       'format', 'salary', 'currency', 'salary_range'], as_index = False).size().sort_values(ascending=False, by = 'size')

ss[ss['size']>1].shape

(327, 13)

In [14]:
df = pd.merge(left = df, right = ss, how = 'inner', validate = 'm:1', on=['company', 'position', 'schedule', 'responsibilities',
       'requirements', 'levels', 'techstack', 'source',
       'format', 'salary', 'currency', 'salary_range'], suffixes = ('', '_r'))[['company', 'position', 'location', 'schedule', 'responsibilities','requirements', 'levels', 'techstack', 'source', 'format', 'salary', 'currency', 'salary_range', 'size']]

In [15]:
df['location'] = df.apply(lambda r: 'Москва' if r['size']>1 else r['location'], axis = 1)

In [16]:
df.drop_duplicates(inplace = True, ignore_index = True)

In [17]:
ss = df.groupby(by = ['company', 'position', 'schedule', 'responsibilities',
       'requirements', 'levels', 'techstack', 'source',
       'format', 'salary', 'currency', 'salary_range'], as_index = False).size().sort_values(ascending=False, by = 'size')

ss[ss['size']>1]

,company,position,schedule,responsibilities,requirements,levels,techstack,source,format,salary,currency,salary_range,size


In [18]:
df.drop(columns=['size'], inplace=True)

После удаления дублей смотрим на распределене зарплат в двух масштабах еще раз:

In [52]:
def salary_range(x):
    step = 10000
    if x == 0:
        return 0
    for i in range(0,100):
        if x > i*step and x <= (i+1)*step:
            return (i+1)*step
        
df['salary_range'] = df['salary'].apply(salary_range)
px.histogram(df[df['salary'] > 0], x = 'salary_range', nbins = 100, color_discrete_sequence = ['steelblue'])

In [53]:
def salary_range(x):
    step = 100000
    if x == 0:
        return 0
    for i in range(0,10):
        if x > i*step and x <= (i+1)*step:
            return (i+1)*step
        
df['salary_range'] = df['salary'].apply(salary_range)
px.histogram(df[df['salary'] > 0], x = 'salary_range', nbins = 10, color_discrete_sequence = ['steelblue'])

In [21]:
lst = ['adas','dsadad']
skill_str = ','.join(lst)
skill_str

'adas,dsadad'

Отработаем метод удаления лишних символов:

In [22]:
df['techstack']

0       ['sql', 'python', 'oracle', 'субд', 'greenplum...
1       ['sql', 'python', 'postgresql', 'spark', 'dock...
2       ['python', 'data science', 'bi', 'ios', 'c', '...
3       ['python', 'data science', 'bi', 'ios', 'c', '...
4       ['python', 'data science', 'bi', 'ios', 'c', '...
                              ...                        
5348                                                   []
5349                                ['python', 'tableau']
5350                            ['java', 'c++', 'oracle']
5351                                           ['pandas']
5352                                     ['etl', 'linux']
Name: techstack, Length: 5353, dtype: object

In [23]:
df['techstack'].apply(lambda x: x.replace('[','').replace(']','').replace('\'',''))

0       sql, python, oracle, субд, greenplum, power bi...
1       sql, python, postgresql, spark, docker, kafka,...
2       python, data science, bi, ios, c, it, r, data,...
3       python, data science, bi, ios, c, r, data, раз...
4       python, data science, bi, ios, c, r, data, раз...
                              ...                        
5348                                                     
5349                                      python, tableau
5350                                    java, c++, oracle
5351                                               pandas
5352                                           etl, linux
Name: techstack, Length: 5353, dtype: object

### 2) Средняя зарплата в разбивке "удаленка\не удаленка"

In [24]:
df['schedule'].nunique()
# schedule format

88

In [25]:
df['format'].nunique()
# schedule format

15

In [26]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [27]:
df['format2'] = df['format'].apply(lambda x: x.replace('[','').replace(']','').replace('\'','').lower().replace('ё','е'))
display(df['format2'].value_counts())

полная занятость              3731
удаленка                       753
не указано                     351
                               258
удаленка, офис                  84
офис                            55
стажировка                      43
частичная занятость             37
проектная работа                15
не имеет значения               13
удаленка, удаленка               6
удаленка, удаленка, офис         4
удаленная работа (на дому)       2
удаленка, гибид                  1
Name: format2, dtype: int64

In [28]:
df[df['source'] == 'hh.ru'][['schedule','format']].value_counts()

schedule          format             
Полный день       Полная занятость       2968
Удаленная работа  Полная занятость        628
Гибкий график     Полная занятость        119
Полный день       Стажировка               26
Удаленная работа  Частичная занятость      24
Сменный график    Полная занятость         13
Удаленная работа  Стажировка                9
Гибкий график     Стажировка                8
Полный день       Частичная занятость       7
Удаленная работа  Проектная работа          7
Гибкий график     Частичная занятость       6
Полный день       Проектная работа          6
Вахтовый метод    Полная занятость          3
Гибкий график     Проектная работа          2
dtype: int64

In [29]:
df[df['source'] != 'hh.ru'][['schedule','format']].value_counts()

schedule                                         format                          
полный                                           удаленка                            639
не указано                                       не указано                          351
[]                                               []                                  151
                                                 ['Удалёнка']                         34
['Fulltime']                                     []                                   21
[]                                               ['Офис']                             18
                                                 ['Удалёнка', 'Офис']                 17
не полный                                        удаленка                             16
['Fulltime']                                     ['Удалёнка']                         16
['Гибкое']                                       []                                   13
Полный рабочий день         

In [30]:
df['format'] = df.apply(lambda r: r['schedule'] if r['source'] == 'hh.ru' else r['format'], axis = 1 )

In [31]:
df['format2'] = df['format'].apply(lambda x: x.replace('[','').replace(']','').replace('\'','').lower().replace('ё','е'))
display(df['format2'].value_counts())

полный день                   3007
удаленка                       753
удаленная работа               668
не указано                     351
                               258
гибкий график                  135
удаленка, офис                  84
офис                            55
не имеет значения               13
сменный график                  13
удаленка, удаленка               6
удаленка, удаленка, офис         4
вахтовый метод                   3
удаленная работа (на дому)       2
удаленка, гибид                  1
Name: format2, dtype: int64

In [32]:
df['format2'] = df['format2'].apply(lambda x: 'удаленка' if x.rfind('удал') >= 0 else ('офис' if x.rfind('офис') >= 0 else 'не указано' ))
display(df['format2'].value_counts())

не указано    3780
удаленка      1518
офис            55
Name: format2, dtype: int64

In [54]:
px.box(df[df['salary'] != 0], x = 'salary', y = 'format2', orientation = 'h', color_discrete_sequence = ['steelblue'])

вывод: работа в офисе и на удаленке оплачивается примерно одинаков, но медиана в офисе на 30 тысяч выше.

In [34]:
# Средняя зарплата в разбивке "удаленка\не удаленка"
# Средняя зарплата в разбивке levels (поле надо преобразовывать!)
# Средняя зарплата в разбивке company

### 3) Средняя зарплата в разбивке по опыту работы

In [35]:
display(df['levels'].value_counts())

df['levels2'] = df['levels'].apply(lambda x: x.replace('[','').replace(']','').replace('\'',''))
display(df['levels2'].value_counts())

def levels_clr(x):
    if x == 'От 1 года до 3 лет': return 'От 1 года до 3 лет'
    if x == 'От 3 до 6 лет': return 'От 3 до 6 лет'
    if x == 'Средний (Middle)': return 'От 1 года до 3 лет'
    if x == 'Старший (Senior)': return 'От 3 до 6 лет'
    if x in ('Нет опыта','Стажёр (Intern)','Без опыта'): return 'От 0 до 1 года'
    if x == 'Более 6 лет': return 'От 6 лет'
    if x in ('Ведущий (Lead)','Lead'): return 'От 6 лет'
    if x == 'Senior': return 'От 3 до 6 лет'
    if x == 'Младший \(Junior\)': return 'От 1 года до 3 лет'
    if x in ('','Not indicated'): return 'не определено'
    else: return 'другое'
    
df['levels2'] = df['levels2'].apply(levels_clr)

display(df['levels2'].value_counts())
    

От 1 года до 3 лет                                   1971
От 3 до 6 лет                                        1472
Средний (Middle)                                      446
Старший (Senior)                                      353
[]                                                    269
Нет опыта                                             242
Более 6 лет                                           141
Not indicated                                         110
Ведущий (Lead)                                         71
['Senior']                                             55
['Middle', 'Senior']                                   43
['Middle']                                             31
Младший (Junior)                                       22
['Lead']                                               14
['Junior']                                             14
['Руководитель']                                       13
['Senior', 'Middle']                                   12
От 1 года     

От 1 года до 3 лет                       1971
От 3 до 6 лет                            1472
Средний (Middle)                          446
Старший (Senior)                          353
                                          269
Нет опыта                                 242
Более 6 лет                               141
Not indicated                             110
Ведущий (Lead)                             71
Senior                                     55
Middle, Senior                             43
Middle                                     31
Младший (Junior)                           22
Lead                                       14
Junior                                     14
Руководитель                               13
Senior, Middle                             12
От 1 года                                  10
Senior, Lead                               10
Junior, Middle, Senior                      6
Junior, Middle                              6
Head                              

От 1 года до 3 лет    2417
От 3 до 6 лет         1880
не определено          379
От 0 до 1 года         248
От 6 лет               226
другое                 203
Name: levels2, dtype: int64

In [55]:
px.box(df[df['salary'] != 0], x = 'salary', y = 'levels2', orientation = 'h',
      category_orders = {'levels2':['От 0 до 1 года','От 1 года до 3 лет','От 3 до 6 лет','От 6 лет','другое','не определено']}, color_discrete_sequence = ['steelblue'])

выводы: 
    - для первых трех уровней медиана удваивается, экспоненциальный рост
    - диапазоны зарплат "от 6 лет" и "3-6 лет" пересекаются на 50% (межквартильный размах переекается)

### 4) Средняя зарплата в разбивке по компаниям

In [37]:
display(df['company'].value_counts())

не определено                        335
СБЕР                                 254
МТС                                   86
VK                                    80
Тинькофф                              78
                                    ... 
АйтиКом                                1
MobiTalents                            1
Крупная горно-добывающая компания      1
ГофроМир                               1
Eclipse Digital Studio                 1
Name: company, Length: 1989, dtype: int64

In [38]:
hunters = df[df['salary'] != 0].groupby(by =['company','levels2'], as_index = False)['salary'].agg(['size', 'mean']).sort_values('mean', ascending = False).reset_index()
hunters[hunters['size'] > 2]

,company,levels2,size,mean
5,Datafold,От 3 до 6 лет,3,595000.000000
42,SberDevices,От 3 до 6 лет,7,357142.857143
55,МТС BigData,другое,3,350000.000000
110,не определено,От 6 лет,8,298750.000000
111,Sportmaster Lab,От 1 года до 3 лет,3,293333.333333
112,RA Clever Recruiting,не определено,3,287000.000000
113,Сбер,От 3 до 6 лет,11,286363.636364
177,не определено,другое,48,249677.083333
178,Quantum Brains,другое,3,246666.666667
184,не определено,не определено,78,238551.564103


In [39]:
df[df['company'] == 'Datafold']

,company,position,location,schedule,responsibilities,requirements,levels,techstack,source,format,salary,currency,salary_range,format2,levels2
4957,Datafold,Python Developer,не определено,"['Fulltime', 'Полная']",не определено,"-Опыт системной разработки от 5 лет, из них ми...",['Senior'],"['python', 'etl', 'sql']",https://t.me/s/datasciencejobs,['Удалёнка'],595000.0,USD,600000,удаленка,От 3 до 6 лет
4969,Datafold,Python Developer,не определено,['Полная'],не определено,"-Опыт разработки от 5 лет, из них минимум 4 го...",['Senior'],"['python', 'etl', 'sql']",https://t.me/s/datasciencejobs,"['Удалёнка', 'Удалёнка']",595000.0,USD,600000,удаленка,От 3 до 6 лет
4972,Datafold,Python Developer,не определено,"['Fulltime', 'Полная']",не определено,"-Опыт разработки от 5 лет, из них минимум 4 го...",['Senior'],"['python', 'etl', 'sql']",https://t.me/s/datasciencejobs,"['Удалёнка', 'Удалёнка']",595000.0,USD,600000,удаленка,От 3 до 6 лет


In [56]:
for lev in list(df['levels2'].unique()):
    fig = px.bar(hunters[(hunters['size'] > 2) & (hunters['levels2'] == lev) ], x = 'company', y = 'mean', 
                 width = 600, height = 600, title = lev, color_discrete_sequence = ['steelblue'])
    fig.show()

Выводы:
    1. больше всего ищут мидлов
    2. в разных категориях опыта разные лидеры
    3. нет лидеров рынка, которые бы набирали много и при этом на большие зарплаты ("пылесосы")
    4. среди компаий лидеров есть малоизветсные компании, чей бизнес ранее не воспринималься как сильно зависимый от аналитики данных

###  5) Средняя зарплата в разбивке по источникам сбора данных

In [57]:
px.bar(df[df['salary'] > 0 ].groupby('source')['salary'].mean(), title = 'Средняя зарплата по ресурсам', color_discrete_sequence = ['steelblue'])

Вывод: чем более специализирован ресурс, тем выше зарплаты.